In [5]:
import numpy as np
import Utilityfunctions as utils
import jax.numpy as jnp
import jax.lax as lax
import ssr_kronvec_jax as ssr
import ssr_likelihood_jax as jax_lik
import ssr_likelihood as pyt
import vanilla as mhn

In [2]:
n = 3
theta = utils.random_theta(n, 0.0)
n_dat = 100
dat = np.random.binomial(n=1, p=0.6, size=n_dat*(2*n+1))
dat = jnp.array(dat).reshape((n_dat, -1))
dat = dat.at[:,-1].set(1)
dat = jnp.column_stack((dat, jnp.ones(n_dat, dtype=int)))
lam1 = 0.5
lam2 = 1.5

In [3]:
#dat_point = dat.at[0,0:2*n+1].get()
#m = dat_point.sum()
#p0 = np.zeros(2**m)
#p0[0] = 1
#pTh1 = jax_lik.R_i_inv_vec(log_theta = theta, x = p0, lam = lam1,  state = dat_point, transpose = False)

In [ ]:
#dat_point = jnp.array([1,1,0,0,1,1,1], dtype = int)
#m = dat_point.sum()
#p0 = np.zeros(2**m)
#p0[0] = 1
#pTh1 = jax_lik.R_i_inv_vec(log_theta = theta, x = p0, lam = lam1,  state = dat_point, transpose = False)
#count = dat_point[1:2*n:2].sum()
#latent_dist = jnp.zeros(2**(count))
#print(utils.ssr_obs_dist(pTh1, dat_point, n, obs_prim=True))
#inds = ssr.obs_inds(pTh1, dat_point, latent_dist, obs_prim=True)
#utils.trunk_states(dat_point)[2**(m-1)+inds[0].at[0].get()]

In [ ]:
#jax_lik.log_prob_coupled(dat, theta, lam1, lam2)

In [ ]:
#jax_lik.log_prob_single(dat, jnp.array(theta), lam1)

In [ ]:
#jax_lik.grad_single(dat, jnp.array(theta), lam1)

In [4]:
state = jnp.array([1,0, 1,0, 0,1, 1]) #dat.at[2, 0:2*n+1].get()
m = int(state.sum())
latent_state = np.append(state.at[1:2*n:2].get(), state.at[-1].get())
count = latent_state.sum()
print(state)
print(latent_state)

latent_dist = jnp.zeros(2**count)
p0 = jnp.zeros(2**m)
p0 = p0.at[0].set(1.)
pTh1 = jax_lik.R_i_inv_vec(log_theta = theta, x = p0, lam = lam1,  state = state, transpose = False)

jax_lik._grad_coupled(theta, lam1, lam2, state, p0, latent_dist, latent_state, True)
#inds = ssr.obs_inds(pTh1, state, latent_dist, obs_prim=True)
#print(utils.trunk_states(state)[inds])
#pTh1_obs = pTh1.at[inds].get()
#print(pTh1_obs)
#iwas = utils.ssr_obs_dist(pTh1, state, n, False)
#print(iwas)
#nk = pTh1_obs.sum()
#q = jnp.zeros_like(p0)
#pos = 2**(m-1) + inds.at[0].get()
#q = q.at[pos].set(1/nk)
#print(q)
#print(utils.ssr_obs_dist(utils.trunk_states(state), state, n, True))
#print(pTh1_obs.shape)
# Actual gradient
#q = ssr.marg_transp(q, state, marg_met=False, marg_seeding=False)
#print(q)
#q = jax_lik.R_i_inv_vec(theta, q, lam1, state, transpose = True)
#g_1 = jax_lik.x_partial_Q_y(theta, q, pTh1, state)
#g_1

[1 0 1 0 0 1 1]
[0 0 1 1]


ValueError: Incompatible shapes for broadcasting: (16,) and requested shape (4,)

In [12]:
prim_first = True
log_theta = theta
n = log_theta.shape[0] - 1
m = np.log2(p0.shape[0]).astype(int)
lm = np.log2(latent_dist.shape[0]).astype(int)

# Calculate joint distributiom at first sampling
pTh1 = lam1*jax_lik.R_i_inv_vec(log_theta, p0, lam1, state)
inds_o = ssr.obs_inds(pTh1, state, latent_dist, prim_first)
pTh1_obs = pTh1.at[inds_o].get()

# calculate q = (pD/pTh_1)^T M
nk = pTh1_obs.sum()
inds_o_t = inds_o.at[inds_o.shape[0//2:]].get()
q = jnp.zeros_like(p0)
q = q.at[inds_o_t].set(1/nk)

# gradient of first summand of the likelihood score
q = jax_lik.R_i_inv_vec(log_theta, q, lam1, state, transpose = True)
g_1 = jax_lik.x_partial_Q_y(log_theta, q, pTh1, state)

# gradients of second term of score
# gradient of pTh_2
pTh1_obs /= nk
log_theta_2 = lax.cond(prim_first,
    lambda th: th,
    lambda th: th.at[:n, -1].set(0.0),
    operand = log_theta)
g_2 = mhn.gradient(log_theta_2, lam2, latent_state, pTh1_obs)
g_2 = lax.cond(prim_first,
    lambda g: g,
    lambda g: g.at[:n, -1].set(0.0),
    operand = g_2)

# gradient of the starting distribution used for calc. pth_2
pTh2 = mhn.R_inv_vec(log_theta, pTh1_obs, lam2,  latent_state, False)
q_2 = jnp.zeros_like(latent_dist)
q_2 = q_2.at[-1].set(1/pTh2.at[-1].get())
q_2 = mhn.R_inv_vec(log_theta, q_2, lam2,  latent_state, True)
q *= 0.0
q = q.at[inds_o].set(q_2)
g_3 = jax_lik.x_partial_Q_y(log_theta, q, pTh1, state)
g_3 /= nk

# gradient of the normalizing sum of the starting distribution used for calc. pth_2
scal = jnp.dot(q, pTh1_obs)
lhs = jnp.zeros_like(p0)
lhs = lhs.at[inds_o].set(1.)
lhs = lam1*jax_lik.R_i_inv_vec(log_theta, lhs, lam1, state, transpose = True)
g4 = jax_lik.x_partial_Q_y(log_theta, lhs, pTh1_obs, state)
g4 *= scal

[  0.       0.       0.     721.8905]
[0 0 1 1]
[ 0.18282016  0.5947506   0.8567908  20.838747  ]
[ 3  7 11 15]
[0. 0. 0. 0.]


TypeError: mul got incompatible shapes for broadcasting: (1, 16), (3, 4).

In [ ]:
mat11 = np.array([[1,0,1,0],[0,1,0,1]])
mat10 = np.array([[1,1]])
mat1 = np.array([[1,0], [0,1]])
matC = np.kron(mat1, np.kron(mat10, np.kron(mat11, mat10)))
B = matC @ pTh1
#print(matC)
print(utils.trunk_states(state))
iwas = np.zeros_like(B)
iwas[-1] = 1/B[-1]
res1 = iwas.T @ matC
print(res1)
iwas2 = np.zeros_like(pTh1)
#iwas2[inds[inds.shape[0]//2:]] = 1/B[-1]
print(iwas2)
theta